<a href="https://colab.research.google.com/github/gitmystuff/AgenticAI/blob/main/09_Pydantic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pydantic AI

In [ ]:
# pip install pydantic-ai logfire nest_asyncio

In [ ]:
import os
import openai
from pydantic import BaseModel
from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic_ai.models.groq import GroqModel
from dotenv import load_dotenv
# from google.colab import userdata
import nest_asyncio
import logfire
logfire.configure(send_to_logfire='if-token-present')

load_dotenv()
# api_key = userdata.get('GROQ_API_KEY')
nest_asyncio.apply()

In [ ]:
model = GroqModel('llama-3.3-70b-versatile')
agent = Agent(model)
result = agent.run_sync("Say Hello")
print(result)

AgentRunResult(output='Hello. How can I assist you today?')


In [ ]:
result.output

'Hello. How can I assist you today?'

In [ ]:
agent

Agent(model=GroqModel(), name='agent', end_strategy='early', model_settings=None, output_type=<class 'str'>, instrument=None)

In [ ]:
import pprint

agent = Agent(model, system_prompt='Be a helpful assistant.')
result = agent.run_sync('Tell me a joke.')
pprint.pprint(result.output)

("Here's a joke for you:\n"
 '\n'
 'What do you call a fake noodle?\n'
 '\n'
 'An impasta!\n'
 '\n'
 'I hope that made you smile! Do you want to hear another one?')


In [ ]:
result2 = agent.run_sync('Explain it please', message_history=result.new_messages())
result2

AgentRunResult(output='The joke is a play on words. "Impasta" sounds similar to "impostor", which means something that is fake or pretending to be something else. But in this case, "impasta" is also a pun on the word "pasta", which is a type of noodle.\n\nSo, the joke is saying that a fake noodle (something that is pretending to be a noodle but isn\'t really one) is called an "impasta" - it\'s a fake pasta, or an impostor noodle. It\'s a lighthearted and silly joke, and the wordplay is what makes it funny!')

In [ ]:
result2.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.', timestamp=datetime.datetime(2025, 10, 19, 15, 55, 9, 845526, tzinfo=datetime.timezone.utc)), UserPromptPart(content='Tell me a joke.', timestamp=datetime.datetime(2025, 10, 19, 15, 55, 9, 845526, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPart(content="Here's a joke for you:\n\nWhat do you call a fake noodle?\n\nAn impasta!\n\nI hope that made you smile! Do you want to hear another one?")], usage=RequestUsage(input_tokens=45, output_tokens=36), model_name='llama-3.3-70b-versatile', timestamp=datetime.datetime(2025, 10, 19, 15, 55, 11, tzinfo=TzInfo(UTC)), provider_name='groq', provider_response_id='chatcmpl-a49598d6-e43f-4379-b926-10f508b0f189'),
 ModelRequest(parts=[UserPromptPart(content='Explain it please', timestamp=datetime.datetime(2025, 10, 19, 15, 55, 24, 667291, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPart(content='The joke is a play on words. "Impasta" sounds similar 

In [ ]:
result2.new_messages()

[ModelRequest(parts=[UserPromptPart(content='Explain it please', timestamp=datetime.datetime(2025, 10, 19, 15, 55, 24, 667291, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPart(content='The joke is a play on words. "Impasta" sounds similar to "impostor", which means something that is fake or pretending to be something else. But in this case, "impasta" is also a pun on the word "pasta", which is a type of noodle.\n\nSo, the joke is saying that a fake noodle (something that is pretending to be a noodle but isn\'t really one) is called an "impasta" - it\'s a fake pasta, or an impostor noodle. It\'s a lighthearted and silly joke, and the wordplay is what makes it funny!')], usage=RequestUsage(input_tokens=94, output_tokens=132), model_name='llama-3.3-70b-versatile', timestamp=datetime.datetime(2025, 10, 19, 15, 55, 26, tzinfo=TzInfo(UTC)), provider_name='groq', provider_response_id='chatcmpl-e905be99-8b26-4d51-b226-c950bfa5b3cf')]

In [ ]:
class HistoricalFigureModel(BaseModel):
    name: str
    era: str
    primary_contribution: str
    # famous_quote: str

agent = Agent(
    model,
    output_type=HistoricalFigureModel
)

result = agent.run_sync('Summarize the inventor of the lightbulb')
print(result.output)

name='Thomas Edison' era='19th-20th century' primary_contribution='invention of the first commercially practical incandescent light bulb'


In [ ]:
class HistoricalFigureModel(BaseModel):
    name: str
    era: str
    primary_contribution: str
    famous_quote: str

agent = Agent(
    model,
    output_type=HistoricalFigureModel
)

result = agent.run_sync('Tell me about the painter of the Mona Lisa')
print(result.output)

name='Leonardo da Vinci' era='Renaissance' primary_contribution='Mona Lisa painting' famous_quote='The noblest pleasure is the joy of understanding'


## Tools

In [ ]:
from typing import Optional
from pydantic import BaseModel

class MyModel(BaseModel):
    task: str
    status: str
    last_task: Optional[str]

agent = Agent(
    model=model,  # Or your small local model
    output_type=MyModel,
    system_prompt="Keep responses short. Handle one task at a time.",
    retries=3
)

last_task = None
last_status = None

@agent.tool
async def add_task(ctx: RunContext, description: str) -> MyModel:
    """Adds a new task."""
    global last_task, last_status
    last_task = description
    last_status = "pending"
    return MyModel(task=description, status="pending", last_task=None)

@agent.tool
async def complete_last(ctx: RunContext) -> MyModel:
    """Marks the last task as completed."""
    global last_task, last_status
    if not last_task:
        raise ValueError("No previous task found.")
    last_status = "done"
    return MyModel(task=last_task, status="done", last_task=last_task)

result = agent.run_sync("Add a task: create slides.")
print(result.output)

result = agent.run_sync("Add a task: review slides.")
print(result.output)

result = agent.run_sync("Now mark the first task as done.")
print(result.output)

result = agent.run_sync("Now mark the last task as done.")
print(result.output)

task='create slides' status='in progress' last_task='create slides'
task='review slides' status='completed' last_task='review slides'
task='Task 1' status='completed' last_task='Task 1'
task='review slides' status='done' last_task='review slides'
